In [1]:
import pandas as pd
from tqdm import tqdm
import transformers
import torch
import torch.nn.functional as F
import numpy as np
import random

/home/rtx4090/miniconda3/envs/ADS/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df_main = pd.read_csv('multitude.csv')
df_main.head()

,text,label,multi_label,split,language,length,source
0,Der Ausbruch des Coronavirus hat die Entwicklu...,1,text-davinci-003,test,de,174,MULTITuDE_MassiveSumm_spiegel
1,Alex Azar was officially sworn in as the U.S. ...,1,text-davinci-003,train,en,57,MULTITuDE_MassiveSumm_voanews
2,Європейський союз вимагає зупинити розтрату ко...,1,gpt-3.5-turbo,test,uk,105,MULTITuDE_MassiveSumm_interfax
3,"Yesterday, hundreds of Zambian university stud...",1,text-davinci-003,train,en,254,MULTITuDE_MassiveSumm_voanews
4,"In a narrow and highly watched vote, the US Se...",1,gpt-4,train,en,416,MULTITuDE_MassiveSumm_voanews


In [3]:
from sklearn.metrics import auc,roc_curve
def get_roc_metrics(human_preds, ai_preds):
    # human_preds is the ai-generated probabiities of human-text
    # ai_preds is the ai-generated probabiities of AI-text
    if not human_preds or not ai_preds:
            # Handle empty arrays to avoid the IndexError
            return None    # Rest of your code
    fpr, tpr, _ = roc_curve([0] * len(human_preds) + [1] * len(ai_preds), human_preds + ai_preds, pos_label=1)
    roc_auc = auc(fpr, tpr)
    return fpr.tolist(), tpr.tolist(), float(roc_auc)

Languages available are : 
- English (en)
- Spanish (es)  
- Russian (ru) 
- Dutch (nl)  
- Catalan (ca)   
- Czech (cs)  
- German (de)   
- Chinese (zh) 
- Portuguese (pt)   
- Arabic (ar) 
- Ukrainian (uk)  


testing on the test split of every language

In [4]:

device = "cuda"# example: cuda:0
detector_path_or_id = "TrustSafeAI/RADAR-Vicuna-7B"
detector = transformers.AutoModelForSequenceClassification.from_pretrained(detector_path_or_id)
tokenizer = transformers.AutoTokenizer.from_pretrained(detector_path_or_id)
detector.eval()
detector.to(device)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=1024, out_features=1024, bias=True)
              (key): Linear(in_features=1024, out_features=1024, bias=True)
              (value): Linear(in_features=1024, out_features=1024, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
 

In [5]:
languages = ['de', 'en', 'es', 'nl', 'pt', 'ru', 'zh', 'ar', 'uk', 'cs', 'ca']
for i in languages:
    df = df_main[df_main['language'] == i]
    df_test = df[df['split'] == 'test']
    df_test_human = df_test[df_test['label'] == 0]
    df_test_ai = df_test[df_test['label'] == 1]
    print('started evaluating for language:', i)
    output_probs_human = []
    output_probs_ai = []
    with torch.no_grad():
        for i  in tqdm(df_test_human['text']):
            inputs = tokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k:v.to(device) for k,v in inputs.items()}
            output_prob = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
            output_probs_human.append(output_prob)

        for i  in tqdm(df_test_ai['text']):
            inputs = tokenizer(i, padding=True, truncation=True, max_length=512, return_tensors="pt")
            inputs = {k:v.to(device) for k,v in inputs.items()}
            output_prob = F.log_softmax(detector(**inputs).logits,-1)[:,0].exp().tolist()
            output_probs_ai.append(output_prob)
    fpr, tpr, roc_auc = get_roc_metrics(output_probs_human, output_probs_ai)
    print('roc_auc:', roc_auc)
    print('fpr:', fpr)
    print('tpr:', tpr)
    
 

started evaluating for language: de


100%|██████████| 2393/2393 [00:26<00:00, 89.14it/s] 


roc_auc: 0.6751097664993216
fpr: [0.0, 0.0, 0.0, 0.003424657534246575, 0.003424657534246575, 0.00684931506849315, 0.00684931506849315, 0.010273972602739725, 0.010273972602739725, 0.0136986301369863, 0.0136986301369863, 0.017123287671232876, 0.017123287671232876, 0.02054794520547945, 0.02054794520547945, 0.023972602739726026, 0.023972602739726026, 0.0273972602739726, 0.0273972602739726, 0.030821917808219176, 0.030821917808219176, 0.03424657534246575, 0.03424657534246575, 0.03767123287671233, 0.03767123287671233, 0.0410958904109589, 0.0410958904109589, 0.04452054794520548, 0.04452054794520548, 0.05136986301369863, 0.05136986301369863, 0.0547945205479452, 0.0547945205479452, 0.05821917808219178, 0.05821917808219178, 0.06164383561643835, 0.06164383561643835, 0.06506849315068493, 0.06506849315068493, 0.0684931506849315, 0.0684931506849315, 0.07191780821917808, 0.07191780821917808, 0.07534246575342465, 0.07534246575342465, 0.07876712328767123, 0.07876712328767123, 0.0821917808219178, 0.08219

100%|██████████| 2214/2214 [00:23<00:00, 92.32it/s] 


roc_auc: 0.8852983475683132
fpr: [0.0, 0.0, 0.0, 0.0036101083032490976, 0.0036101083032490976, 0.007220216606498195, 0.007220216606498195, 0.010830324909747292, 0.010830324909747292, 0.01444043321299639, 0.01444043321299639, 0.018050541516245487, 0.018050541516245487, 0.021660649819494584, 0.021660649819494584, 0.02527075812274368, 0.02527075812274368, 0.02888086642599278, 0.02888086642599278, 0.032490974729241874, 0.032490974729241874, 0.036101083032490974, 0.036101083032490974, 0.039711191335740074, 0.039711191335740074, 0.04332129963898917, 0.04332129963898917, 0.04693140794223827, 0.04693140794223827, 0.05054151624548736, 0.05054151624548736, 0.05415162454873646, 0.05415162454873646, 0.05776173285198556, 0.05776173285198556, 0.061371841155234655, 0.061371841155234655, 0.06498194945848375, 0.06498194945848375, 0.06859205776173286, 0.06859205776173286, 0.07220216606498195, 0.07220216606498195, 0.07581227436823104, 0.07581227436823104, 0.07942238267148015, 0.07942238267148015, 0.08303

...

roc_auc: 0.5415951632543257
fpr: [0.0, 0.0, 0.0, 0.003355704697986577, 0.003355704697986577, 0.006711409395973154, 0.006711409395973154, 0.010067114093959731, 0.010067114093959731, 0.013422818791946308, 0.013422818791946308, 0.016778523489932886, 0.016778523489932886, 0.020134228187919462, 0.020134228187919462, 0.02348993288590604, 0.02348993288590604, 0.026845637583892617, 0.026845637583892617, 0.030201342281879196, 0.030201342281879196, 0.03355704697986577, 0.03355704697986577, 0.040268456375838924, 0.040268456375838924, 0.04697986577181208, 0.04697986577181208, 0.050335570469798654, 0.050335570469798654, 0.053691275167785234, 0.053691275167785234, 0.05704697986577181, 0.05704697986577181, 0.06040268456375839, 0.06040268456375839, 0.06375838926174497, 0.06375838926174497, 0.06711409395973154, 0.06711409395973154, 0.07046979865771812, 0.07046979865771812, 0.07718120805369127, 0.07718120805369127, 0.08053691275167785, 0.08053691275167785, 0.08389261744966443, 0.08389261744966443, 0.087

100%|██████████| 2389/2389 [00:30<00:00, 78.50it/s]


roc_auc: 0.7005776475512767
fpr: [0.0, 0.0, 0.0, 0.0033333333333333335, 0.0033333333333333335, 0.006666666666666667, 0.006666666666666667, 0.01, 0.01, 0.013333333333333334, 0.013333333333333334, 0.016666666666666666, 0.016666666666666666, 0.02, 0.02, 0.023333333333333334, 0.023333333333333334, 0.02666666666666667, 0.02666666666666667, 0.03, 0.03, 0.03333333333333333, 0.03333333333333333, 0.03666666666666667, 0.03666666666666667, 0.04, 0.04, 0.043333333333333335, 0.043333333333333335, 0.04666666666666667, 0.04666666666666667, 0.05, 0.05, 0.05333333333333334, 0.05333333333333334, 0.056666666666666664, 0.056666666666666664, 0.06, 0.06, 0.06333333333333334, 0.06333333333333334, 0.06666666666666667, 0.06666666666666667, 0.07, 0.07, 0.07333333333333333, 0.07333333333333333, 0.07666666666666666, 0.07666666666666666, 0.08, 0.08, 0.08333333333333333, 0.08333333333333333, 0.08666666666666667, 0.08666666666666667, 0.09, 0.09, 0.09333333333333334, 0.09333333333333334, 0.09666666666666666, 0.096666

100%|██████████| 2391/2391 [00:31<00:00, 77.03it/s]

roc_auc: 0.6443147915795343
fpr: [0.0, 0.0, 0.0, 0.0033333333333333335, 0.0033333333333333335, 0.006666666666666667, 0.006666666666666667, 0.01, 0.01, 0.013333333333333334, 0.013333333333333334, 0.016666666666666666, 0.016666666666666666, 0.02, 0.02, 0.023333333333333334, 0.023333333333333334, 0.02666666666666667, 0.02666666666666667, 0.03, 0.03, 0.03333333333333333, 0.03333333333333333, 0.03666666666666667, 0.03666666666666667, 0.04, 0.04, 0.043333333333333335, 0.043333333333333335, 0.04666666666666667, 0.04666666666666667, 0.05, 0.05, 0.05333333333333334, 0.05333333333333334, 0.056666666666666664, 0.056666666666666664, 0.06, 0.06, 0.06333333333333334, 0.06333333333333334, 0.07, 0.07, 0.07333333333333333, 0.07333333333333333, 0.07666666666666666, 0.07666666666666666, 0.08, 0.08, 0.08333333333333333, 0.08333333333333333, 0.08666666666666667, 0.08666666666666667, 0.09333333333333334, 0.09333333333333334, 0.09666666666666666, 0.09666666666666666, 0.1, 0.1, 0.10333333333333333, 0.10333333

AttributeError: 'list' object has no attribute 'mean'